## Task 1
#### Question: 
Build and populate necessary tables
#### Core Code with expected output:

In [1]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
import pandas as pd

spark = SparkSession.builder \
    .appName("fifa") \
    .config("spark.driver.extraClassPath", "/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/pyspark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

data_sample = pd.read_csv("./archive/players_15.csv", nrows=2)
column_names = list(data_sample.columns)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/29 14:54:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def infer_sql_type(dtype):
    """Map pandas data type to PostgreSQL"""
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    else:
        return "VARCHAR"

# Infer data type for each column in Postgres
column_types = {col: infer_sql_type(dtype) for col, dtype in zip(data_sample.columns, data_sample.dtypes)}

# Create a new column of year
column_types['year'] = "INT"

# Print the crrent result
for col, col_type in column_types.items():
    print(f"{col}: {col_type}")

sofifa_id: INT
player_url: VARCHAR
short_name: VARCHAR
long_name: VARCHAR
player_positions: VARCHAR
overall: INT
potential: INT
value_eur: FLOAT
wage_eur: FLOAT
age: INT
dob: VARCHAR
height_cm: INT
weight_kg: INT
club_team_id: FLOAT
club_name: VARCHAR
league_name: VARCHAR
league_level: INT
club_position: VARCHAR
club_jersey_number: INT
club_loaned_from: FLOAT
club_joined: VARCHAR
club_contract_valid_until: INT
nationality_id: INT
nationality_name: VARCHAR
nation_team_id: FLOAT
nation_position: VARCHAR
nation_jersey_number: INT
preferred_foot: VARCHAR
weak_foot: INT
skill_moves: INT
international_reputation: INT
work_rate: VARCHAR
body_type: VARCHAR
real_face: VARCHAR
release_clause_eur: FLOAT
player_tags: VARCHAR
player_traits: VARCHAR
pace: INT
shooting: INT
passing: INT
dribbling: INT
defending: INT
physic: INT
attacking_crossing: INT
attacking_finishing: INT
attacking_heading_accuracy: INT
attacking_short_passing: INT
attacking_volleys: INT
skill_dribbling: INT
skill_curve: INT
skil

In [3]:
table = "fifa" 
columns = ",\n    ".join([f'"{col}" {dtype}' for col, dtype in column_types.items()])
create_table = f"""
CREATE TABLE IF NOT EXISTS {table} (
    id SERIAL PRIMARY KEY,
    {columns}
);
"""

print(create_table)


CREATE TABLE IF NOT EXISTS fifa (
    id SERIAL PRIMARY KEY,
    "sofifa_id" INT,
    "player_url" VARCHAR,
    "short_name" VARCHAR,
    "long_name" VARCHAR,
    "player_positions" VARCHAR,
    "overall" INT,
    "potential" INT,
    "value_eur" FLOAT,
    "wage_eur" FLOAT,
    "age" INT,
    "dob" VARCHAR,
    "height_cm" INT,
    "weight_kg" INT,
    "club_team_id" FLOAT,
    "club_name" VARCHAR,
    "league_name" VARCHAR,
    "league_level" INT,
    "club_position" VARCHAR,
    "club_jersey_number" INT,
    "club_loaned_from" FLOAT,
    "club_joined" VARCHAR,
    "club_contract_valid_until" INT,
    "nationality_id" INT,
    "nationality_name" VARCHAR,
    "nation_team_id" FLOAT,
    "nation_position" VARCHAR,
    "nation_jersey_number" INT,
    "preferred_foot" VARCHAR,
    "weak_foot" INT,
    "skill_moves" INT,
    "international_reputation" INT,
    "work_rate" VARCHAR,
    "body_type" VARCHAR,
    "real_face" VARCHAR,
    "release_clause_eur" FLOAT,
    "player_tags" VARCHAR,

In [16]:
# Male dataset
total_data = spark.read.csv('/Users/jiayi/Desktop/System-ToolChain-AI/Course Project/archive/players_15.csv',header=True, inferSchema=True).toDF(*column_names)
total_data = total_data.withColumn("year", lit(2015))

for i in range(16,23):
    data = spark.read.csv(f'/Users/jiayi/Desktop/System-ToolChain-AI/Course Project/archive/players_{i}.csv',header=True, inferSchema=True)
    data = data.toDF(*column_names)
    data = data.withColumn("year", lit(2000+i))
    total_data = total_data.union(data)
print("total male players: ",total_data.count())

# Female dataset
for i in range(16,23):
    data = spark.read.csv(f'/Users/jiayi/Desktop/System-ToolChain-AI/Course Project/archive/female_players_{i}.csv',header=True, inferSchema=True)
    data = data.toDF(*column_names)
    data = data.withColumn("year", lit(2000+i))
    total_data = total_data.union(data)
    
print("total male+female players: ",total_data.count())   
# data_total = data_train.union(data_test)
# total_data.printSchema()

# total_data.count()


# total_data = total_data.limit(10).toPandas()
# display(total_data) 

total male players:  142079
total male+female players:  144323


In [17]:
total_data.distinct().count()

144323

In [19]:
from pyspark.sql import SparkSession
postgres_url = "jdbc:postgresql://localhost:5432/jiayi"
postgres_properties = {
    "user": "jiayi", 
    "password": "Liu123jiayi",
    "driver": "org.postgresql.Driver"
}

total_data.write \
    .jdbc(url=postgres_url, table="fifa", mode="overwrite", properties=postgres_properties)

data_postgres = spark.read \
    .jdbc(url=postgres_url, table="fifa", properties=postgres_properties)

data_postgres.printSchema()
data_postgres.show(5, vertical=True)

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: string (nullable = true)
 |-- wage_eur: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: string (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: string (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: string (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: string (nullable = true)
 |-- nationality_id: integer (nullable = true)
 |

In [21]:
data_postgres.tail(5)

[Row(sofifa_id=262371, player_url='https://sofifa.com/player/262371/craig-dundas/220002', short_name='C. Dundas', long_name='Craig Dundas', player_positions='ST', overall=56, potential=56, value_eur=None, wage_eur='1000.0', age=40, dob=datetime.date(1981, 2, 16), height_cm=188, weight_kg=89, club_team_id='110799.0', club_name='Sutton United', league_name='English League Two', league_level='4', club_position='RES', club_jersey_number='14', club_loaned_from=None, club_joined='2019-07-01', club_contract_valid_until='2022', nationality_id=14, nationality_name='England', nation_team_id=None, nation_position=None, nation_jersey_number=None, preferred_foot='Right', weak_foot=3, skill_moves=2, international_reputation=1, work_rate='Medium/Low', body_type='Normal (185+)', real_face='No', release_clause_eur=None, player_tags=None, player_traits=None, pace=50, shooting=56, passing=39, dribbling=53, defending=23, physic=63, attacking_crossing=34, attacking_finishing=58, attacking_heading_accuracy=

In [24]:
data_postgres.filter(data_postgres.sofifa_id == 158023).show(vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | RW                   
 overall                     | 93                   
 potential                   | 93                   
 value_eur                   | 8.9E7                
 wage_eur                    | 575000.0             
 age                         | 29                   
 dob                         | 1987-06-24           
 height_cm                   | 170                  
 weight_kg                   | 72                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | RW             

## Task II

#### Task 2 --Q1
#### Question: 
In Year X, what were the Y clubs that had the highest number of players with
contracts ending in year Z (or after)?
- X is a year between (2015 and 2022, inclusively).
- Y is a positive integer.
- Z is a year that can hold the value of 2023 or a year after it.
#### Core Code with expected output:

In [52]:
data_postgres.select("club_contract_valid_until").distinct().show()

+-------------------------+
|club_contract_valid_until|
+-------------------------+
|                     2016|
|                     2020|
|                     2031|
|                     2026|
|                     2019|
|                     2017|
|                     2014|
|                     2028|
|                     2018|
|                     2022|
|                     2025|
|                     2023|
|                     2021|
|                     2024|
|                     2015|
|                     2027|
|                     NULL|
+-------------------------+



In [89]:
# Possible values for Z: 2031, 2026, 2028, 2025, 2023, 2024, 2027
# data_postgres.filter('club_contract_valid_until >= 2023').filter('year==2018').distinct().show(vertical=True)

In [91]:
def query_no_clubs(X, Z):
    # Filter the data as needed and group by club_name
    club_counts = data_postgres.filter(f"club_contract_valid_until == {Z}") \
                               .filter(f"year == {X}") \
                               .groupBy("club_name") \
                               .count()

    # Find the maximum count value
    max_count = club_counts.agg(F.max("count")).collect()[0][0]
    
    # Count the number of clubs that have the maximum count value
    no_of_clubs = club_counts.filter(F.col("count") == max_count).count()
    return int(no_of_clubs)

#### Function Testing:

In [92]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("contract_ending", IntegerType(), True),
    StructField("no_of_clubs", IntegerType(), True)
])

# Create an empty DataFrame with the defined schema
resultDF = spark.createDataFrame([], schema=schema)


data =[]
X = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Z = [2031, 2026, 2028, 2025, 2023, 2024, 2027]

for i in X:
    for j in Z:
        no_of_clubs = query_no_clubs(i,j)
        data.append((int(i), int(j), int(no_of_clubs)))

In [93]:
# Answer
resultDF = spark.createDataFrame(data, schema=schema)
resultDF.filter("no_of_clubs > 0").show()

+----+---------------+-----------+
|year|contract_ending|no_of_clubs|
+----+---------------+-----------+
|2017|           2023|          1|
|2018|           2026|          2|
|2018|           2023|          1|
|2019|           2026|          2|
|2019|           2025|          5|
|2019|           2023|          1|
|2019|           2024|          1|
|2020|           2026|          1|
|2020|           2025|          1|
|2020|           2023|          2|
|2020|           2024|          2|
|2021|           2026|          1|
|2021|           2028|          1|
|2021|           2025|          4|
|2021|           2023|         18|
|2021|           2024|          1|
|2021|           2027|          1|
|2022|           2031|          1|
|2022|           2026|          1|
|2022|           2028|          1|
+----+---------------+-----------+
only showing top 20 rows



In [84]:
# testing the max players in club when contract=2023 & year=2021
club_counts = data_postgres.filter(f"club_contract_valid_until == 2023") \
                           .filter(f"year == 2021") \
                           .groupBy("club_name") \
                           .count().show()
max_count = club_counts.agg(F.max("count")).collect()[0][0]
print(max_count)

20


In [85]:
# Count and validate the number of clubs with players=20 (manual check!)
club_counts = data_postgres.filter(f"club_contract_valid_until == 2023") \
                           .filter(f"year == 2021") \
                           .groupBy("club_name") \
                           .count() \
                           .collect()

for row in club_counts:
    print(row)

Row(club_name='Shonan Bellmare', count=2)
Row(club_name='FC Voluntari', count=5)
Row(club_name='Göztepe SK', count=3)
Row(club_name='Salford City', count=1)
Row(club_name='Guangzhou FC', count=9)
Row(club_name='1. FC Union Berlin', count=4)
Row(club_name='Sagan Tosu', count=6)
Row(club_name='Jiangsu FC', count=3)
Row(club_name='Puebla FC', count=7)
Row(club_name='Shandong Taishan', count=6)
Row(club_name='Argentinos Juniors', count=7)
Row(club_name='San Lorenzo de Almagro', count=8)
Row(club_name='SpVgg Unterhaching', count=5)
Row(club_name='Club Athletico Paranaense', count=20)
Row(club_name='FC Luzern', count=4)
Row(club_name='Universidad Católica', count=1)
Row(club_name='Fortaleza', count=20)
Row(club_name='SC Heerenveen', count=6)
Row(club_name='AIK', count=5)
Row(club_name='Sandefjord Fotball', count=1)
Row(club_name='Delfín Sporting Club', count=2)
Row(club_name='Santos', count=20)
Row(club_name='AC Mineros de Guayana', count=1)
Row(club_name='Hibernian', count=6)
Row(club_name=

#### Task 2 --Q2
#### Question: 
In sports, maturity and energy of teams depend on the average age of team players (among other factors). Therefore, it’s important to have a function that can find clubs with such features.
- List the X clubs with the highest (or lowest) average player age for a given year Y.
    - X represents a positive integer, but you should handle a scenario if X is not positive value.
    - Y represents a year between 2015 and 2022 inclusively.
    - Provide the user with the ability to choose if they want the highest average age or the lowest average age.
    - Make sure to handle this scenario as well: if the user requests 5 clubs with highest averages but there are 3 clubs that share the same count at rank number 5, please include all of them in your output
#### Core Code with expected output:

In [140]:
def top_avg_age_clubs(X, Y, filter):
    avg_by_group = data_postgres.filter(f"year == {Y}") \
                            .groupBy("club_name") \
                            .agg(F.avg("age").alias("average_age"))
    
    # Find the distinct avg age
    avg_by_group.select("average_age").distinct().orderBy("average_age")
    
    if filter == "highest":
        top = avg_by_group.select("average_age").distinct().sort("average_age", ascending=False).limit(X).collect()
        top_boundary = top[X-1][0]
        return avg_by_group.filter(f"average_age >= {top_boundary}").sort("average_age", ascending=False).show()
    else:
        bottom = avg_by_group.select("average_age").distinct().sort("average_age", ascending=True).limit(X).collect()
        bottom_boundary = bottom[X-1][0]
        return avg_by_group.filter(f"average_age <= {bottom_boundary}").sort("average_age", ascending=True).show()
        
    

In [141]:
top_avg_age_clubs(5, 2015, "highest")

+--------------------+------------------+
|           club_name|       average_age|
+--------------------+------------------+
|           Cruz Azul|28.071428571428573|
|        Arsenal Tula|             28.04|
|Podbeskidzie Biel...|27.962962962962962|
|       Fenerbahçe SK|             27.88|
|Leones Negros de ...| 27.79310344827586|
+--------------------+------------------+



#### Task 2 --Q3
#### Question: 
What is the most popular nationality in the dataset for each year? (i.e. display the
most frequent nation for 2015, 2016, etc.).
#### Core Code with expected output:

In [162]:
for i in range(2015,2023):
    no_nations = data_postgres.filter(f"year == {i}") \
                         .groupBy("nationality_name") \
                         .count()
    
    
    max_count = no_nations.agg(F.max("count")).collect()[0][0]
    print(f"The most popularity nationality & its total players in year {i}:")
    no_of_nations = no_nations.filter(F.col("count") == max_count).show()


The most popularity nationality & its total players in year 2015:
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1627|
+----------------+-----+

The most popularity nationality & its total players in year 2016:
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1540|
+----------------+-----+

The most popularity nationality & its total players in year 2017:
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1650|
+----------------+-----+

The most popularity nationality & its total players in year 2018:
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1656|
+----------------+-----+

The most popularity nationality & its total players in year 2019:
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1648|
+----------------+-----+

The most popularity nationality & its to

In [163]:
# Test
for row in no_nations.collect():
    print(row)

Row(nationality_name="Côte d'Ivoire", count=111)
Row(nationality_name='Chad', count=2)
Row(nationality_name='Russia', count=93)
Row(nationality_name='Paraguay', count=197)
Row(nationality_name='Chinese Taipei', count=2)
Row(nationality_name='Senegal', count=127)
Row(nationality_name='Sweden', count=408)
Row(nationality_name='Guyana', count=7)
Row(nationality_name='Philippines', count=6)
Row(nationality_name='Eritrea', count=1)
Row(nationality_name='Malaysia', count=2)
Row(nationality_name='Fiji', count=1)
Row(nationality_name='Turkey', count=291)
Row(nationality_name='Malawi', count=2)
Row(nationality_name='Iraq', count=4)
Row(nationality_name='Germany', count=1237)
Row(nationality_name='Comoros', count=8)
Row(nationality_name='Afghanistan', count=1)
Row(nationality_name='Jordan', count=3)
Row(nationality_name='Sudan', count=3)
Row(nationality_name='France', count=1003)
Row(nationality_name='Greece', count=98)
Row(nationality_name='Kosovo', count=46)
Row(nationality_name='Montserrat', 